<a href="https://colab.research.google.com/github/SAIKARTHIGEYAN1512/SNLP/blob/master/word_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

In [2]:
embedding_layer = layers.Embedding(1000, 5)

In [3]:
result = embedding_layer(tf.constant([1,2,3]))
result.numpy()

array([[ 0.01295639, -0.01742864,  0.00978468, -0.01784465, -0.044724  ],
       [-0.04566658, -0.04795233,  0.01705867,  0.04895754, -0.04112607],
       [ 0.02776045, -0.02332877,  0.01878455, -0.0283838 ,  0.04917992]],
      dtype=float32)

In [4]:
result = embedding_layer(tf.constant([[0,1,2],[3,4,5]]))
result.shape

TensorShape([2, 3, 5])

In [5]:
(train_data, test_data), info = tfds.load(
    'imdb_reviews/subwords8k', 
    split = (tfds.Split.TRAIN, tfds.Split.TEST), 
    with_info=True, as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incomplete0ZLTSE/imdb_reviews-train.tfrecord
Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incomplete0ZLTSE/imdb_reviews-test.tfrecord
Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incomplete0ZLTSE/imdb_reviews-unsupervised.tfrecord
Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [6]:
encoder = info.features['text'].encoder
encoder.subwords[:20]

['the_',
 ', ',
 '. ',
 'a_',
 'and_',
 'of_',
 'to_',
 's_',
 'is_',
 'br',
 'in_',
 'I_',
 'that_',
 'this_',
 'it_',
 ' /><',
 ' />',
 'was_',
 'The_',
 'as_']

In [7]:
train_batches = train_data.shuffle(1000).padded_batch(10)
test_batches = test_data.shuffle(1000).padded_batch(10)

In [8]:
train_batch, train_labels = next(iter(train_batches))
train_batch.numpy()

array([[2383, 1782, 4013, ...,    0,    0,    0],
       [  12,   31,  456, ...,    0,    0,    0],
       [  19, 2031,   18, ...,    0,    0,    0],
       ...,
       [1342,   15,   56, ...,    0,    0,    0],
       [  62,   27,    9, ...,    0,    0,    0],
       [  12, 5705, 7961, ...,    0,    0,    0]])

In [9]:
embedding_dim=16

model = keras.Sequential([
  layers.Embedding(encoder.vocab_size, embedding_dim),
  layers.GlobalAveragePooling1D(),
  layers.Dense(16, activation='relu'),
  layers.Dense(1)
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 16)          130960    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 131,249
Trainable params: 131,249
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(
    train_batches,
    epochs=10,
    validation_data=test_batches, validation_steps=20)

Epoch 1/10
2500/2500 [==============================] - 12s 5ms/step - loss: 0.5118 - accuracy: 0.6966 - val_loss: 0.4272 - val_accuracy: 0.8250
Epoch 2/10
2500/2500 [==============================] - 12s 5ms/step - loss: 0.2843 - accuracy: 0.8847 - val_loss: 0.3669 - val_accuracy: 0.8400
Epoch 3/10
2500/2500 [==============================] - 11s 4ms/step - loss: 0.2292 - accuracy: 0.9092 - val_loss: 0.4808 - val_accuracy: 0.8050
Epoch 4/10
2500/2500 [==============================] - 11s 4ms/step - loss: 0.1995 - accuracy: 0.9228 - val_loss: 0.3896 - val_accuracy: 0.8600
Epoch 5/10
2500/2500 [==============================] - 11s 4ms/step - loss: 0.1764 - accuracy: 0.9339 - val_loss: 0.3828 - val_accuracy: 0.8650
Epoch 6/10
2500/2500 [==============================] - 12s 5ms/step - loss: 0.1578 - accuracy: 0.9420 - val_loss: 0.3692 - val_accuracy: 0.9000
Epoch 7/10
2500/2500 [==============================] - 12s 5ms/step - loss: 0.1448 - accuracy: 0.9467 - val_loss: 0.4881 - val_ac

In [12]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) 

(8185, 16)


In [15]:
import io

encoder = info.features['text'].encoder

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for num, word in enumerate(encoder.subwords):
  vec = weights[num+1] 
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
out_v.close()
out_m.close()